# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [1]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import SimpleReader
import numpy as np
import h5py

import sys
import json
import requests
from IPython.display import IFrame

## Start the server

In [3]:
%%bash --bg --proc bg_proc
python /Users/agurvich/research/repos/Firefly/FireflyFlaskApp.py

## Show Firefly in an IFrame

In [12]:
url = "http://localhost:5000"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [59]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("dummy.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to dummy.hdf5")

np.savetxt('dummy.csv',coords)
print("output to dummy.csv")

## test out the csv
simple = SimpleReader("dummy.csv",write_jsons_to_disk=False,extension='.csv')

## test out the hdf5
simple = SimpleReader("dummy.hdf5",write_jsons_to_disk=False,extension='.hdf5')

output to dummy.hdf5
output to dummy.csv
Opening 1 files and 1 particle types...
JSONdir is None, defaulting to /Users/agurvich/research/repos/Firefly/static/data/Data
outputting: 
ParticleGroup: dummy
Contains 7999 particles (7999 after decimation) and 0 tracked fields
Opening 1 files and 1 particle types...
JSONdir is None, defaulting to /Users/agurvich/research/repos/Firefly/static/data/Data
outputting: 
ParticleGroup: group1
Contains 8000 particles (8000 after decimation) and 0 tracked fields


## Send this data to the Flask app

In [6]:
JSON_array = simple.dumpToJSON(write_jsons_to_disk=False)

outputting: 
ParticleGroup: group1
Contains 8000 particles (8000 after decimation) and 0 tracked fields


Opening 1 files and 6 particle types...
JSONdir is None, defaulting to /Users/agurvich/research/repos/Firefly/static/data/Data
outputting: 
ParticleGroup: PartType0
Contains 1066668 particles (10666 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType1
Contains 5860827 particles (58608 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType2
Contains 6175377 particles (61753 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType3
Contains 7732157 particles (77321 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType4
Contains 514118 particles (5141 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType5
Contains 1 particles (1 after decimation) and 0 tracked fields


In [35]:
reader = SimpleReader("/users/agurvich/research/snaps/isolated_disks/Control_G4_20/snapdir_050",
    write_jsons_to_disk=False,
    decimation_factor=100)
print(len(reader.JSON))

Opening 1 files and 6 particle types...
JSONdir is None, defaulting to /Users/agurvich/research/repos/Firefly/static/data/Data
outputting: 
ParticleGroup: PartType0
Contains 1066668 particles (10666 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType1
Contains 5860827 particles (58608 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType2
Contains 6175377 particles (61753 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType3
Contains 7732157 particles (77321 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType4
Contains 514118 particles (5141 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType5
Contains 1 particles (1 after decimation) and 0 tracked fields
> /Users/agurvich/research/repos/Firefly/dataReader/firefly_api/reader.py(284)dumpToJSON()
    282             ##  we want to send dataViaFlask
    283             import pdb; pdb.set_trace()
--> 284             self.JS

In [8]:
reader.dumpToJSON(write_jsons_to_disk=False)

outputting: 
ParticleGroup: PartType0
Contains 1066668 particles (10666 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType1
Contains 5860827 particles (58608 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType2
Contains 6175377 particles (61753 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType3
Contains 7732157 particles (77321 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType4
Contains 514118 particles (5141 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType5
Contains 1 particles (1 after decimation) and 0 tracked fields


In [21]:
JSON_arrays[2][1][0:50]

'{"Coordinates":[[-204.3753051758,134.370010376,-12'

In [4]:
import time
init = time.time()
foo = reader.outputToDict(JSON=True)
print(time.time()-init,'s elapsed')

20.48990797996521 s elapsed


In [6]:
foo[:100]

'{"parts":{"PartType0":{"Coordinates":[[-8.290719986,6.6803894043,-12.6101322174],[-7.0236144066,7.12'

*This sends the data.  (Then look back above to see it.)*

In [11]:
simple.sendDataViaFlask()

posting...data posted!


In [13]:
## alternatively, make the request yourself (literally just copying the code from sendDataViaFlask())
port = 5000
outputDict = simple.outputToDict(json_friendly=True)
requests.post(f'http://localhost:{port:d}/data_input',json=json.dumps(outputDict))

<Response [200]>

*To kill the Firefly server.*

In [9]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill


kill: 29931: No such process
